In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from os import getcwd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau

In [2]:
train_dir = 'C:/Users/devbh/COVID19 datasets - Limited (3 classes)/Training'
validation_dir = 'C:/Users/devbh/COVID19 datasets - Limited (3 classes)/Testing'

train_pneu_dir = train_dir+'/PNEUMONIA'+'/train/'
train_covid_dir = train_dir+'/COVID19'+'/train/'
train_normal_dir = train_dir+'/NORMAL'+'/train/'

validation_pneu_dir = validation_dir+'/PNEUMONIA'+'/test/'
validation_covid_dir = validation_dir+'/COVID19'+'/test/'
validation_normal_dir = validation_dir+'/NORMAL'+'/test/'

In [3]:
train_covid_dir

'C:/Users/devbh/COVID19 datasets - Limited (3 classes)/Training/COVID19/train/'

In [4]:
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.95):
            print("\nReached limit accuracy so cancelling training!")
            self.model.stop_training = True

In [5]:
ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

In [6]:
batch = 12

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size = batch,
                                                   class_mode = 'categorical',
                                                   target_size = (224, 224),
                                                   shuffle = True)     

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                        batch_size = batch,
                                                        class_mode = 'categorical',
                                                        target_size = (224, 224),
                                                        shuffle = True)

Found 750 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [7]:
local_weights_file = f"{getcwd()}/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"

pre_trained_model = VGG19(include_top = False, 
                          weights = None,
                          input_shape = (224, 224, 3))

pre_trained_model.load_weights(local_weights_file)
  
pre_trained_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
for layer in pre_trained_model.layers[:-3]:
    layer.trainable = False
for layer in pre_trained_model.layers[-3:]:
    layer.trainable = True 

In [9]:
last_layer = pre_trained_model.get_layer('block5_conv4')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 14, 14, 512)


In [10]:
x = AveragePooling2D((4, 4)) (last_output)
x = Flatten()(x)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.5)(x)                  
x = Dense(3, activation = 'softmax')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(learning_rate = 0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
callbacks = myCallback()
reduce_lr = ReduceLROnPlateau()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    verbose = 1,
                    epochs = 40,
                    batch_size = batch,
                    callbacks = [callbacks, reduce_lr])

Epoch 1/40
63/63 [==============================] - 254s 4s/step - loss: 1.0914 - accuracy: 0.6613 - val_loss: 0.4028 - val_accuracy: 0.8200 - lr: 0.0010
Epoch 2/40
63/63 [==============================] - 305s 5s/step - loss: 0.3432 - accuracy: 0.8840 - val_loss: 0.2478 - val_accuracy: 0.9200 - lr: 0.0010
Epoch 3/40
63/63 [==============================] - 246s 4s/step - loss: 0.2527 - accuracy: 0.9200 - val_loss: 0.1700 - val_accuracy: 0.9400 - lr: 0.0010
Epoch 4/40
63/63 [==============================] - 266s 4s/step - loss: 0.1672 - accuracy: 0.9520 - val_loss: 0.1473 - val_accuracy: 0.9467 - lr: 0.0010
Epoch 5/40
63/63 [==============================] - 306s 5s/step - loss: 0.1374 - accuracy: 0.9613 - val_loss: 0.2917 - val_accuracy: 0.8933 - lr: 0.0010
Epoch 6/40
63/63 [==============================] - ETA: 0s - loss: 0.1337 - accuracy: 0.9627
Reached limit accuracy so cancelling training!
63/63 [==============================] - 355s 6s/step - loss: 0.1337 - accuracy: 0.9627 -

In [12]:
results = model.evaluate(validation_generator, batch_size=12)
print("test loss, test acc:", results)

13/13 [==============================] - 65s 5s/step - loss: 0.1661 - accuracy: 0.9600
test loss, test acc: [0.16613057255744934, 0.9599999785423279]


In [13]:
len(pre_trained_model.layers)

22